# CNN (Convolution Neural Netwrok)
CNN is a popular deep learning model in dealing with images. We implement CNN with pytorch to compare the performance with the multi-layer model. We will also implement CNN inkeras in other file to make comparison.

# Load Data
Note that we normalize the training data by dividing the each pixel value with 255.

In [1]:
import mnist
import torch
import torch.nn as nn
import torch.tensor as tensor
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from time import time
x_train, t_train, x_test, t_test = mnist.load()
xtrain=tensor(x_train/255,dtype=torch.float)
ttrain=tensor(t_train,dtype=torch.int64)
xtest=tensor(x_test/255,dtype=torch.float)
ttest=tensor(t_test,dtype=torch.int64)

plt.imshow(xtrain[1].view(28,28))
xtrain=xtrain.view(-1,1,28,28)
xtest=xtest.view(-1,1,28,28)
print(xtrain.shape,xtest.shape)

torch.Size([60000, 1, 28, 28]) torch.Size([10000, 1, 28, 28])


# Set the Model
We use CNN (convolution neural network) to predict the data.
* Conv1: We use **Conv2d(1,6,5)** to build our first convolution layer. Since our digit image is not colorful, it only have one channel. We set the first parameter to one as the inital channel of given image. We set our output channel to six, and set our kernel size to five (i.e., each filter kernel is 5*5)

* Conv2: we call **Conv2d(6,16,5)** which accept six-channel input to builf second convolution layer. We set the out channel to sixteen and the kernal size is five (5*5 kernel).

* MaxPooling: After each convolution layer, we will call **MaxPool2d(2,2)** to do the max pooling. We set the size of the window (2,2) to take a max over.

 

In the Net.forward() we set our procedure.
* We first apply *conv1*, call the relu (rectified linear unit) funtion to filter the unnecessary features, and then apply max pooling.
* We then apply *conv2*, call the relu function, and then apply max pooling.
* We transform the dataset to have 256 columns which can fit in the the fisrt linear layer *fc1*, and then apply *tanh* 
* We fit the data to second linear layer *fc2*. we markout *softmax* since pytorch builds softmax inside their cross entropy loss funciton.

Note that we set the first linear layer transform data with 256 features. Since sfter we fit the data into conv1, max pooling, conv2, and max pooling again, the data shape became (16,4,4) where 16 denotes the channel number and other two denote the width and height. So after we flatten the data, the data size became 256(4*4*16).

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(256, 120)#(28-4)/2=12,(12-4)/2=4(size=4*4), there are 16 filters
        self.fc2 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,256)
        x = torch.relu(self.fc1(x))
        #x=F.softmax(self.fc2(x),dim=1) #softmax is built in cross entrophy loss function
        x=self.fc2(x)
        return x
net = Net()

# Model training 
* CrossEntropyLoss: cross entropy loss in pytorch already encoded softmax. so we mark out the softmax function in our model (above cell).

* optimizer: We first try the SGD(Stochastic gradient descent) with momentun, and the result is 0.916. Then we use optimer Adam, an other popular methed for a method for stochastic optimization, and had better accuray upto 0.975. Both of them take around 7 seconds.

* We run only 1 epoch, and divide the data into batches.
    * We extract batch *x* from training data and batch *t* labels.
    * **optimizer.zero_grad** is to set the gradients to zero. It can avoid mixing the result with previous epoch.
    * we fit the training data (batch *x*) to our model, and have the output y
    * we compute the cross entropy loss with our output y and the training labels (batch *t*)
    * we call **loss.backward** to run the back propagation to compute the error derivatives for model parameters
    * we call **optimizer.step** to update the model parameters with the result from back propagation.
    
Note that we try to change the batch sizes. Larger batch size may run faster but not necessary have better perofrmance. In some case, batch size too large consumes too much memory and slow down the speed. So users can choose different size depend on the data size and computer load.

In [3]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr = 0.01)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

stime=time()
tsize=xtrain.shape[0] #Total size of the data.
bsize=300 #Batch size of the data. We also try 30 or 2000.
for epoch in range(1):
    for i in range(int(tsize/bsize)):
            x=xtrain[bsize*i:bsize*(i+1)]
            t=ttrain[bsize*i:bsize*(i+1)]
            
            optimizer.zero_grad()
            y = net(x)
            loss = criterion(y, t)
            
            loss.backward()
            optimizer.step()
print(loss.item())
print('time',time()-stime)


0.22494377195835114
time 7.541453838348389


# Evaluation 
We fit the testing data(xtest) into our mode, and print out the result and compare with the testing labels. And got the final accuracy 0.975.
Note that tuning the parameters like learning rate, batch size or epochs is important to accelerate the training process or improve the accuracy.

In [4]:
ypred=net(xtest.view(-1,1,28,28))

print(ypred.max(1)[1])
print(ttest)

float((ttest==ypred.max(1)[1]).sum())/len(ttest)

tensor([7, 2, 1,  ..., 4, 5, 6])
tensor([7, 2, 1,  ..., 4, 5, 6])


0.9753

# Compare the multi-layer network
We find that while multi-layer network need 4-5 epoch to get the more than 0.97 accuracy while CNN can perform the same with only one epoch. But multi-layer only takes 1-2 seconds while CNN takes 7 seconds. 
We guess the difference is because CNN involves complicated computaion which lower down the speed but can achieve high accuracy while only run the data once.

In conclude, since the images in MNIST is not too complicated, multi-layer network can have good performance with better efficiency than CNN. But for complicated images, CNN should be a better way to adopt for its sophiscated computation,